In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 4.4 MB 35.0 MB/s 


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
train_df = pd.read_csv('/content/train_data.csv')
test_df = pd.read_csv('/content/test_data.csv')
print(train_df.shape,test_df.shape)

(7613, 5) (3263, 4)


In [ ]:
train_df = train_df.drop(['id','keyword','location'],axis=1)
target = train_df['target']
train_df = train_df.drop('target',axis=1)
test_df = test_df.drop(['id','keyword','location'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_df,target,test_size = 0.20,random_state = 42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(6090, 1) (1523, 1) (6090,) (1523,)


In [ ]:
#---------------Use_bert_to_preprocess_and_generate_embedding-------------------
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:

def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:

# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [17]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
191/191 [==============================] - 2342s 12s/step - loss: 0.6298 - accuracy: 0.6406 - precision: 0.6015 - recall: 0.4893
Epoch 2/10
191/191 [==============================] - 2331s 12s/step - loss: 0.5956 - accuracy: 0.6890 - precision: 0.6594 - recall: 0.5744
Epoch 3/10
191/191 [==============================] - 2336s 12s/step - loss: 0.5837 - accuracy: 0.7010 - precision: 0.6762 - recall: 0.5862
Epoch 4/10
191/191 [==============================] - 2333s 12s/step - loss: 0.5702 - accuracy: 0.7138 - precision: 0.6898 - recall: 0.6091
Epoch 5/10
191/191 [==============================] - 2318s 12s/step - loss: 0.5633 - accuracy: 0.7161 - precision: 0.6909 - recall: 0.6163
Epoch 6/10
191/191 [==============================] - 2319s 12s/step - loss: 0.5542 - accuracy: 0.7284 - precision: 0.7079 - recall: 0.6285
Epoch 7/10
191/191 [==============================] - 2322s 12s/step - loss: 0.5488 - accuracy: 0.7307 - precision: 0.7124 - recall: 0.6281
Epoch 8/10
191/191 [

In [1]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

NameError: ignored

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [20]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.69      0.95      0.80       874
           1       0.87      0.42      0.57       649

    accuracy                           0.73      1523
   macro avg       0.78      0.69      0.68      1523
weighted avg       0.77      0.73      0.70      1523

              precision    recall  f1-score   support

           0       0.69      0.95      0.80       874
           1       0.87      0.42      0.57       649

    accuracy                           0.73      1523
   macro avg       0.78      0.69      0.68      1523
weighted avg       0.77      0.73      0.70      1523

